In [ ]:
import requests
from lxml import etree as et

ns = {
    'srw': 'http://www.loc.gov/zing/srw/'
}
sru_url = 'https://obv-at-oenb.alma.exlibrisgroup.com/view/sru/43ACC_ONB'
p = {
    'version': '1.2',
    'operation': 'searchRetrieve',
    'maximumRecords': 50
}
max_records = 50

In [ ]:
params = p.copy()
query = 'alma.national_bibliography_number=OeBC and alma.local_field_970=56 and alma.main_pub_date>2009'
params['query'] = query
params['startRecord'] = 1
marc_results = []

while params['startRecord']:
    res = requests.get(sru_url, params=params)
    root = et.fromstring(res.content)
    marc_results += root.xpath('//srw:records/srw:record', namespaces=ns)
    start_record = root.find('.//srw:nextRecordPosition', namespaces=ns)
    num_results = int(root.find('.//srw:numberOfRecords', namespaces=ns).text)
    if start_record != None:
        start_record = int(start_record.text)
    print(params['startRecord'], num_results, res.url)
    params['startRecord'] = start_record

results = et.Element('results')
[results.append(r) for r in marc_results]

et.ElementTree(results).write('marc_results.xml')